In [3]:
spark.sql('DROP TABLE txkt.dim_video_df')

DataFrame[]

In [5]:
spark.sql(
    '''
        CREATE TABLE IF NOT EXISTS txkt.dim_video_df
         (
          `video_id`             STRING COMMENT '视频ID',
          `video_title`          STRING COMMENT '视频标题',
          `video_type1`          STRING COMMENT '视频大类',
          `video_type2`          STRING COMMENT '视频中类',
          `video_type3`          STRING COMMENT '视频小类',
          `video_url`            STRING COMMENT '视频url',
          `organ_name`           STRING COMMENT '视频所属机构名称',
          `video_sections_num`   BIGINT COMMENT '视频节数',
          `video_index_page`     BIGINT COMMENT '视频所在页数',
          `price`                BIGINT COMMENT '收费信息(单位：分)',
          `person_num`           BIGINT COMMENT '购买/报名人数',
          `recently_study_num`   BIGINT COMMENT '最近在学',
          `sign_up_num`          BIGINT COMMENT '累计报名',
          `video_praise_degree`  DOUBLE COMMENT '好评度',
          `etl_time`             STRING COMMENT '数据加工时间'
        ) 
        PARTITIONED BY (`dt` STRING) 
        STORED AS PARQUET
    '''
)


DataFrame[]

In [35]:
spark.sql(
    'INSERT OVERWRITE TABLE txkt.dim_video_df PARTITION (dt)' + 
    '''
        SELECT SPLIT(video_url, '/')[4] AS video_id,
               video_title, 
               video_type1,        
               video_type2,         
               video_type3,                 
               video_url,           
               organ_name, 
               CAST(CASE WHEN video_sections_num IS NOT NULL THEN video_sections_num ELSE 0 END AS BIGINT) AS video_sections_num,  
               video_index_page,  
               CAST(CASE WHEN price              IS NOT NULL THEN price              ELSE 0 END AS BIGINT) AS price,  
               CAST(CASE WHEN person_num         IS NOT NULL THEN person_num         ELSE 0 END AS BIGINT) AS person_num,  
               CAST(CASE WHEN recently_study_num IS NOT NULL THEN recently_study_num ELSE 0 END AS BIGINT) AS recently_study_num,  
               CAST(CASE WHEN sign_up_num IS NOT NULL THEN REPLACE(recently_study_num, '万', '0000') ELSE 0 END AS BIGINT) AS sign_up_num,  
               CASE WHEN video_praise_degree > '0' THEN CAST(REPLACE(video_praise_degree, '%', '') AS INT)/100 ELSE 0 END  AS video_praise_degree,            
               FROM_UNIXTIME(CAST(NOW() AS BIGINT), 'yyyy-MM-dd HH:mm:ss') AS elt_time,
               DATE_FORMAT(TO_TIMESTAMP(grab_time, 'yyyy-MM-dd HH:mm:ss'), 'yyyy-MM-dd') AS dt 
          FROM ( 
                 SELECT video_title,  
                        video_type1,       
                        video_type2,         
                        video_type3,                 
                        video_url,           
                        organ_name, 
                        video_sections_num,  
                        video_index_page,            
                        price,               
                        person_num,          
                        recently_study_num,  
                        sign_up_num,         
                        video_praise_degree,
                        grab_time,
                        ROW_NUMBER() OVER(PARTITION BY video_url ORDER BY grab_time DESC) AS rk
                   FROM txkt.ods_tencent_study_video
                  WHERE dt='{dt}' 
               ) AS a
         WHERE rk=1
    '''.format(dt='2022-03-26')
)

DataFrame[]

In [19]:
spark.sql("SELECT * FROM txkt.dim_video_df WHERE dt='2022-03-26' LIMIT 10;").toPandas()

,video_id,video_title,video_type1,video_type2,video_type3,video_url,organ_name,video_sections_num,video_index_page,price,person_num,recently_study_num,sign_up_num,video_praise_degree,etl_time,dt
0,1101435,mid函数：Excel表格制作中截取字符串指定中间字符,职业·职场,办公软件,Excel,https://ke.qq.com/course/1101435,韩志华,1,25,500,1,0,0,0.0,2022-03-26 10:15:59,2022-03-25
1,114742,windows2008虚拟化Hyper-V课程,IT·互联网,运维,其他,https://ke.qq.com/course/114742,韩利辉,47,16,39900,19,0,0,0.0,2022-03-26 10:15:59,2022-03-25
2,1184146,第二册：汉语口语速成基础篇,语言·留学,小语种,多语种,https://ke.qq.com/course/1184146,古丽孜然·太来提,50,5,9900,1,0,0,0.0,2022-03-26 10:15:59,2022-03-25
3,1236191,淘宝美工入门(VIP试听）,设计·创作,平面设计,电商设计,https://ke.qq.com/course/1236191,常德市计算机职业培训学校,2,10,0,1,2,2,0.0,2022-03-26 10:15:59,2022-03-25
4,1293510,塑胶齿轮结合kisssoft设计,设计·创作,环境艺术设计,建筑设计,https://ke.qq.com/course/1293510,罗宾老师,1,4,0,6,27,27,1.0,2022-03-26 10:15:59,2022-03-25
5,1293989,设计狗/图像合成/修图/PS教程合成/海报/调色/坚果产品海报合成,设计·创作,平面设计,平面视觉,https://ke.qq.com/course/1293989,设计狗,2,30,500,1,0,0,0.0,2022-03-26 10:15:59,2022-03-25
6,1294033,书法/硬笔书法全网书写效果改变最快最明显【网格习字】同步辅导,兴趣·生活,生活百科,书法,https://ke.qq.com/course/1294033,北京一画教育咨询有限责任公司,21,7,689800,3,0,0,0.0,2022-03-26 10:15:59,2022-03-25
7,1346464,计算机一级等级考试（维语）,IT·互联网,认证考试,计算机等级考试,https://ke.qq.com/course/1346464,信心电子,6,16,2500,4,0,0,0.0,2022-03-26 10:15:59,2022-03-25
8,1347420,Linux-RHCSA入门精讲之磁盘管理（7下）,IT·互联网,运维,Linux运维,https://ke.qq.com/course/1347420,新盟教育,1,12,0,2,3,3,0.0,2022-03-26 10:15:59,2022-03-25
9,1347537,速卖通/跨境电商/全球开网店,电商·营销,跨境电商,速卖通,https://ke.qq.com/course/1347537,好学微客电商教育,2,3,0,1,14,14,0.0,2022-03-26 10:15:59,2022-03-25


In [57]:
spark.sql(
    '''
    SELECT video_id, COUNT(*) AS cnt
      FROM txkt.dim_video_df
     WHERE dt='2022-03-24'
  GROUP BY video_id
    HAVING cnt>1
    '''
).show()

+--------+---+
|video_id|cnt|
+--------+---+
+--------+---+



In [21]:
spark.sql(
    '''
    SELECT '' > '0', '1%' > '0'
    '''
).show()

+------+--------+
|( > 0)|(1% > 0)|
+------+--------+
| false|    true|
+------+--------+



In [58]:
spark.sql(
    '''
    SELECT SPLIT(video_url, '/')[4], video_url
      FROM txkt.dim_video_df
     WHERE dt='2022-03-24'
    '''
).show(2, False)

+--------------------------+--------------------------------+
|split(video_url, /, -1)[0]|video_url                       |
+--------------------------+--------------------------------+
|https:                    |https://ke.qq.com/course/1103217|
|https:                    |https://ke.qq.com/course/1129012|
+--------------------------+--------------------------------+
only showing top 2 rows

